<a href="https://colab.research.google.com/github/KIQVinicius/Data-Science/blob/master/AnomaliasJob.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
from datetime import timedelta  

In [0]:
import numpy as np
from datetime import date

In [0]:
dataset = pd.read_csv("MIPS1.csv")

In [0]:
porcentagem = float(dataset['Porcentagem'][1])/100 + 1
dataset.columns = ['Index', 'JOBNAME', 'DATE', 'Valor_MIPS', 'Porcentagem', 'Chave']

In [0]:
dataset = dataset.drop(columns=["Porcentagem"], axis=1)

In [0]:
dataset['DATE'] = pd.to_datetime(dataset['DATE'])

In [0]:
first_day = dataset['DATE'].min()
last_day = dataset['DATE'].max()

In [8]:
first_day

Timestamp('2019-01-11 00:00:00')

In [9]:
last_day

Timestamp('2020-12-05 00:00:00')

In [0]:
dataset_first_day = dataset.query("DATE == " +"'" + str(first_day) + "'")
dataset_last_day = dataset.query("DATE == " +"'" + str(last_day) + "'")

In [0]:
dt = last_day.date() - first_day.date()

In [0]:
weeks = dt.days/7

In [0]:
weeks = int(weeks)

In [14]:
weeks

99

In [0]:
data = first_day

In [16]:
data+ timedelta(days = 7)

Timestamp('2019-01-18 00:00:00')

In [17]:
data = first_day
tabela_final = dataset.iloc[0:0]
soma = 0
dataset_semana_anterior = dataset.query("DATE >= " + " '"+ str(data)+ "' " + " and DATE < " + " '"+str(data+ timedelta(days = 7)) + "' ")
data = data+ timedelta(days = 7)
dataset_semana_anterior = dataset_semana_anterior[dataset_semana_anterior['Valor_MIPS']>0].groupby('JOBNAME')
dataset_semana_anterior = pd.DataFrame(dataset_semana_anterior['Valor_MIPS'].mean())
dataset_semana_anterior.reset_index(inplace =True)
for week in range(0, weeks, 1):
  dataset_semana = dataset.query("DATE >= " + " '"+ str(data)+ "' " + " and DATE < " + " '"+str(data+ timedelta(days = 7)) + "' ")
  dataset = dataset.query("DATE >= " + " '"+ str(data+ timedelta(days = 7))+ "' ")
  print("Dataset: " + str(dataset.shape[0]))
  print("Final: " + str(tabela_final.shape[0]))
  soma = soma + dataset_semana.shape[0]
  dataset_semana = dataset_semana[dataset_semana['Valor_MIPS']>0].groupby('JOBNAME')
  dataset_semana = pd.DataFrame(dataset_semana['Valor_MIPS'].mean())
  dataset_semana.reset_index(inplace=True)
  for index, row in dataset_semana.iterrows():
    tabela_final = tabela_final.append(dataset_semana_anterior.query("JOBNAME == "+"'"+row['JOBNAME']+"'" + " and Valor_MIPS > "+ str(float(row['Valor_MIPS']) * porcentagem)))
  dataset_semana_anterior = dataset_semana
  data = data+ timedelta(days = 7)
dataset_ultima_semana = dataset.query("Index >= " + str(soma))

Dataset: 1117393
Final: 0
Dataset: 1117393
Final: 0
Dataset: 1117393
Final: 0
Dataset: 1108518
Final: 0
Dataset: 1108518
Final: 0
Dataset: 1108518
Final: 0
Dataset: 1108518
Final: 0
Dataset: 1096482
Final: 0
Dataset: 1096482
Final: 0
Dataset: 1096482
Final: 0
Dataset: 1096482
Final: 0
Dataset: 1086794
Final: 0
Dataset: 1081723
Final: 0
Dataset: 1081723
Final: 1071
Dataset: 1081723
Final: 1071
Dataset: 1081723
Final: 1071
Dataset: 1067016
Final: 1071
Dataset: 1067016
Final: 1071
Dataset: 1067016
Final: 1071
Dataset: 1067016
Final: 1071
Dataset: 1054278
Final: 1071
Dataset: 1054278
Final: 1071
Dataset: 1054278
Final: 1071
Dataset: 1054278
Final: 1071
Dataset: 1044518
Final: 1071
Dataset: 1040463
Final: 1071
Dataset: 1040463
Final: 2437
Dataset: 1040463
Final: 2437
Dataset: 1040463
Final: 2437
Dataset: 1027628
Final: 2437
Dataset: 1027628
Final: 2437
Dataset: 1027628
Final: 2437
Dataset: 1027628
Final: 2437
Dataset: 1013560
Final: 2437
Dataset: 1013560
Final: 2437
Dataset: 1013560
Final: 